In [1]:
import keras
import tensorflow as tf


config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 10} ) 
sess = tf.Session(config=config) 
keras.backend.set_session(sess)

Using TensorFlow backend.


In [2]:
print("Tensor Flow Version: {}".format(tf.__version__))

Tensor Flow Version: 1.13.1


## Classification

In [3]:
import pandas as pd
import io
import requests
import numpy as np
from sklearn import metrics
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.callbacks import EarlyStopping

In [5]:
df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/iris.csv", 
    na_values=['NA', '?'])

# Convert to numpy - Classification
x = df[['sepal_l', 'sepal_w', 'petal_l', 'petal_w']].values
dummies = pd.get_dummies(df['species']) # Classification
species = dummies.columns
y = dummies.values

# Split into validation and training sets
x_train, x_test, y_train, y_test = train_test_split(    
    x, y, test_size=0.25, random_state=42)

# Build neural network
model = Sequential()
model.add(Dense(50, input_dim=x.shape[1], activation='relu')) # Hidden 1
model.add(Dense(25, activation='relu')) # Hidden 2
model.add(Dense(y.shape[1],activation='softmax')) # Output
model.compile(loss='categorical_crossentropy', optimizer='adam')

monitor = EarlyStopping(monitor='val_loss', 
                        min_delta=1e-3, 
                        patience=5, #number of epochs with no improvement after which training will be stopped.
                        verbose=1, 
                        mode='auto',
                        restore_best_weights=True
                       )
model.fit(x_train,y_train,
          validation_data=(x_test,y_test),
          callbacks=[monitor],
          verbose=2,
          epochs=1000)

Instructions for updating:
Use tf.cast instead.
Train on 112 samples, validate on 38 samples
Epoch 1/1000
 - 1s - loss: 1.0498 - val_loss: 1.0414
Epoch 2/1000
 - 0s - loss: 0.9917 - val_loss: 0.9949
Epoch 3/1000
 - 0s - loss: 0.9678 - val_loss: 0.9608
Epoch 4/1000
 - 0s - loss: 0.9365 - val_loss: 0.9306
Epoch 5/1000
 - 0s - loss: 0.9089 - val_loss: 0.9123
Epoch 6/1000
 - 0s - loss: 0.8921 - val_loss: 0.8909
Epoch 7/1000
 - 0s - loss: 0.8669 - val_loss: 0.8540
Epoch 8/1000
 - 0s - loss: 0.8454 - val_loss: 0.8297
Epoch 9/1000
 - 0s - loss: 0.8253 - val_loss: 0.8096
Epoch 10/1000
 - 0s - loss: 0.8038 - val_loss: 0.7913
Epoch 11/1000
 - 0s - loss: 0.7905 - val_loss: 0.7716
Epoch 12/1000
 - 0s - loss: 0.7668 - val_loss: 0.7489
Epoch 13/1000
 - 0s - loss: 0.7513 - val_loss: 0.7280
Epoch 14/1000
 - 0s - loss: 0.7308 - val_loss: 0.7080
Epoch 15/1000
 - 0s - loss: 0.7137 - val_loss: 0.6969
Epoch 16/1000
 - 0s - loss: 0.7003 - val_loss: 0.6735
Epoch 17/1000
 - 0s - loss: 0.6777 - val_loss: 0.647

Epoch 147/1000
 - 0s - loss: 0.0897 - val_loss: 0.0710
Epoch 148/1000
 - 0s - loss: 0.0880 - val_loss: 0.0719
Epoch 149/1000
 - 0s - loss: 0.0897 - val_loss: 0.0776
Epoch 150/1000
 - 0s - loss: 0.0859 - val_loss: 0.0704
Epoch 151/1000
 - 0s - loss: 0.0869 - val_loss: 0.0709
Epoch 152/1000
 - 0s - loss: 0.0855 - val_loss: 0.0696
Epoch 153/1000
 - 0s - loss: 0.0847 - val_loss: 0.0714
Epoch 154/1000
 - 0s - loss: 0.0834 - val_loss: 0.0764
Epoch 155/1000
 - 0s - loss: 0.0851 - val_loss: 0.0742
Epoch 156/1000
 - 0s - loss: 0.0831 - val_loss: 0.0690
Epoch 157/1000
 - 0s - loss: 0.0839 - val_loss: 0.0678
Epoch 158/1000
 - 0s - loss: 0.0901 - val_loss: 0.0673
Epoch 159/1000
 - 0s - loss: 0.0839 - val_loss: 0.0733
Epoch 160/1000
 - 0s - loss: 0.0834 - val_loss: 0.0808
Epoch 161/1000
 - 0s - loss: 0.0850 - val_loss: 0.0732
Epoch 162/1000
 - 0s - loss: 0.0814 - val_loss: 0.0671
Restoring model weights from the end of the best epoch
Epoch 00162: early stopping


In [6]:
from sklearn.metrics import accuracy_score

pred = model.predict(x_test)
predict_classes = np.argmax(pred,axis=1)
expected_classes = np.argmax(y_test,axis=1)
correct = accuracy_score(expected_classes,predict_classes)
print(f"Accuracy: {correct}")

Accuracy: 0.9736842105263158


## Regression

In [7]:
df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/auto-mpg.csv", 
    na_values=['NA', '?'])

cars = df['name']

# Handle missing value
df['horsepower'] = df['horsepower'].fillna(df['horsepower'].median())

# Pandas to Numpy
x = df[['cylinders', 'displacement', 'horsepower', 'weight',
       'acceleration', 'year', 'origin']].values
y = df['mpg'].values # regression

# Split into validation and training sets
x_train, x_test, y_train, y_test = train_test_split(    
    x, y, test_size=0.25, random_state=42)

# Build the neural network
model = Sequential()
model.add(Dense(25, input_dim=x.shape[1], activation='relu')) # Hidden 1
model.add(Dense(10, activation='relu')) # Hidden 2
model.add(Dense(1)) # Output
model.compile(loss='mean_squared_error', optimizer='adam')

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto',
        restore_best_weights=True)
model.fit(x_train,y_train,
          validation_data=(x_test,y_test),
          callbacks=[monitor],verbose=2,epochs=1000)

Train on 298 samples, validate on 100 samples
Epoch 1/1000
 - 0s - loss: 133319.1233 - val_loss: 74561.6619
Epoch 2/1000
 - 0s - loss: 59004.5239 - val_loss: 40118.5272
Epoch 3/1000
 - 0s - loss: 30929.2186 - val_loss: 20519.2127
Epoch 4/1000
 - 0s - loss: 15532.6567 - val_loss: 10110.0346
Epoch 5/1000
 - 0s - loss: 7663.1073 - val_loss: 5100.4461
Epoch 6/1000
 - 0s - loss: 3909.4979 - val_loss: 2696.3969
Epoch 7/1000
 - 0s - loss: 2100.7387 - val_loss: 1755.6884
Epoch 8/1000
 - 0s - loss: 1471.6494 - val_loss: 1290.4503
Epoch 9/1000
 - 0s - loss: 893.4462 - val_loss: 265.6893
Epoch 10/1000
 - 0s - loss: 245.0630 - val_loss: 166.4855
Epoch 11/1000
 - 0s - loss: 178.0845 - val_loss: 106.4734
Epoch 12/1000
 - 0s - loss: 132.3489 - val_loss: 109.6739
Epoch 13/1000
 - 0s - loss: 126.9353 - val_loss: 109.7395
Epoch 14/1000
 - 0s - loss: 126.6941 - val_loss: 107.3153
Epoch 15/1000
 - 0s - loss: 137.4892 - val_loss: 145.8468
Epoch 16/1000
 - 0s - loss: 133.0432 - val_loss: 102.3151
Epoch 17/1

In [8]:
# Measure RMSE error.  RMSE is common for regression.
pred = model.predict(x_test)
score = np.sqrt(metrics.mean_squared_error(pred,y_test))
print(f"Final score (RMSE): {score}")

Final score (RMSE): 7.921030120694821
